# Image filtering
### Author: Pawel Budzynski

In [ ]:
import numpy as np
import skimage.io
import skimage.color
import matplotlib.pyplot as plt

## Gaussian filter

To generate values in each cell of the kernel the equation is used:
$$g(x, y) = \frac{1}{2\pi\sigma^2}e^{-\frac{x^2 + y^2}{2\sigma^{2}}}$$

In [ ]:
def gaussian_filter(
    image: np.ndarray,
    size: int = 3,
) -> np.ndarray:
    """Apply Gaussian filter of requested size to a grayscale image."""
    
    if size%2 == 0:
        raise TypeError("Kernel size must be an odd integer.")
    
    # Generate Gaussian kerner of the requested size.
    x, y = np.meshgrid(
        np.linspace(-1, 1, size), 
        np.linspace(-1, 1, size),
    )
    kernel = (1/(2*np.pi))*np.exp(-(x**2 + y**2)/2)
    
    # Get margin respecting the kernel size.
    m = size//2
    # Empty matrix to store results.
    new_img = np.zeros(image.shape)
    # Apply zero padding so result image is the same size as original.
    image = np.pad(image, m)
    # Iteratively apply kernel to every pixel.
    for i in range(new_img.shape[0]):
        for j in range(new_img.shape[1]):
            new_img[i][j] = np.sum(image[i:i+2*m+1,j:j+2*m+1] * kernel)
    
    return new_img

## Sobel filter
Sobel filter to define as applying two filters to get the horizontal and vertical derivative approximations respectively. Assuming $I$ is the input image and the operator is convolution operation:
\begin{equation}
G_x =
\begin{bmatrix}
1 & 0 & -1\\
2 & 0 & -2 \\
1 & 0 & -1
\end{bmatrix}\ast I,\;\;
G_y = 
\begin{bmatrix}
1 & 2 & 1\\
0 & 0 & 0 \\
-1 & -2 & -1
\end{bmatrix}\ast I,
\end{equation}
then compute the result as a gradient magnitute
$$G = \sqrt{{G_x}^2 + {G_y}^2}. $$

In [ ]:
def sobel_filter(image: np.ndarray) -> np.ndarray:
    """Apply Sobel filter to a grayscale image."""
    # Horizontal derivative kernel.
    gx = np.array([
        [1, 0, -1],
        [2, 0, -2],
        [1, 0, -1],
    ])
    # Vertical derivarive kernel.
    gy = np.array([
        [1, 2, 1],
        [0, 0, 0],
        [-1, -2, -1],
    ])
    # Matrices to store partial results.
    x_image = np.zeros(image.shape)
    y_image = np.zeros(image.shape)
    # Apply zero padding so output image has the same size as original.
    image = np.pad(image, 1)
    # Iteratively apply filters to every pixel.
    for i in range(x_image.shape[0]):
        for j in range(x_image.shape[1]):
            x_image[i][j] = np.sum(image[i:i+3,j:j+3] * gx)
            y_image[i][j] = np.sum(image[i:i+3,j:j+3] * gy)
    
    # Compute gradient magnitude.
    return np.sqrt(x_image**2 + y_image**2)

## Averaging filter
The filter adds up values covered by the kernel and then compute an average. In practice it might be considered as applying following convolution (assuming kernel size equals 3 as an example):

\begin{equation}
K = \frac{1}{9}
\begin{bmatrix}
1 & 1 & 1\\
1 & 1 & 1 \\
1 & 1 & 1
\end{bmatrix}.
\end{equation}

In [ ]:
def avg_filter(
    image: np.ndarray, 
    size: int = 3,
) -> np.ndarray:
    """Apply averaging filter of a requested size to a grayscale image."""
    
    if size%2 == 0:
        raise TypeError("Kernel size must be an odd integer.")
    
    # Empty matrix to store results.
    new_img = np.zeros(image.shape)
    # Get margin respecting kernel size.
    m = size//2
    # Apply zero padding so the output image has the same size as original.
    image = np.pad(image, m)
    # Iteratively sum values a pixel and its neighbours for every pixel.
    for i in range(new_img.shape[0]):
        for j in range(new_img.shape[1]):
            new_img[i][j] = np.sum(image[i:i+2*m+1,j:j+2*m+1])
    # Get average value by simply divided by amount of pixels added.
    return new_img / (size*size)

## Gradient filter
Compute gradient approximation using finite differences. For horizontal and vertical gradient it is realized by appllying following filters:
\begin{equation}
G_x =\begin{bmatrix}
1 \\
0 \\
-1
\end{bmatrix}
,\;\;
G_y = 
\begin{bmatrix}
1 & 0 & -1
\end{bmatrix}.
\end{equation}

In [ ]:
def gradient_filter(
    image: np.ndarray, 
    axis: str = 'x',
) -> np.ndarray:
    """Apply gradient filter to a grayscale along given axis."""
    
    # Empty matrix to store results.
    new_image = np.zeros(image.shape)
    if axis == 'x':
        # Apply zero padding to preserve original image size.
        image = np.pad(image, ((1,1), (0,0)))
        # For each row sum values from the row above and the row below.
        for i in range(new_image.shape[0]):
            new_image[i] = image[i+2] - image[i]
    elif axis == 'y':
        # Apply zero padding to preserve original image size.
        image = np.pad(image, ((0,0), (1,1)))
        # For each column sum values from the column to the left and to the right.
        for j in range(new_image.shape[1]):
            new_image[:,j] = image[:,j+2] - image[:,j]
    return new_image

In [ ]:
image = skimage.io.imread('barbara.jpg')
image = skimage.color.rgb2gray(image)

In [ ]:
new_image = gaussian_filter(image, size=11)
print("Output shape: ", new_image.shape)
plt.imshow(new_image, cmap='gray')
plt.title("Gaussian filter.")
plt.show()

In [ ]:
new_image = sobel_filter(image)
print("Output shape: ", new_image.shape)
plt.imshow(new_image, cmap='gray')
plt.title("Sobel filter.")
plt.show()

In [ ]:
new_image = avg_filter(image, size=7)
print("Output shape: ", new_image.shape)
plt.imshow(new_image, cmap='gray')
plt.title("Averaging filter.")
plt.show()

In [ ]:
new_image = gradient_filter(image, axis='x')
print("Output shape: ", new_image.shape)
plt.imshow(new_image, cmap='gray')
plt.title("Gradient filter along x axis.")
plt.show()

In [ ]:
new_image = gradient_filter(image, axis='y')
print("Output shape: ", new_image.shape)
plt.imshow(new_image, cmap='gray')
plt.title("Gradient filter along y axis.")
plt.show()

Compare Sobel filter result with example on Wikipedia: https://en.wikipedia.org/wiki/Sobel_operator

In [ ]:
image2 = skimage.io.imread('image.png')
image2 = skimage.color.rgb2gray(image2)
new_image2 = sobel_filter(image2)

print("Original shape:", image2.shape)
print("Output shape: ", new_image2.shape)

fig = plt.figure(figsize=(50, 200))
plt.subplot(1, 2, 1)
plt.imshow(image2, cmap='gray')
plt.subplot(1, 2, 2)
plt.imshow(new_image2, cmap='gray')
plt.show()